In [2]:
df = pd.read_csv('2TA_final.csv',index_col=False)

In [3]:
# df = pd.read_pickle('clustering_aux.pkl')

In [2]:
df = pd.concat([pd.read_csv('1whole_dish_base_2020.csv',index_col=False),
                pd.read_csv('2whole_dish_base_2020.csv',index_col=False)])

In [2]:
#df

### add category


In [5]:
len(cat)

NameError: name 'cat' is not defined

In [7]:
cat = pd.read_csv('catego.csv')
cat['item_code'] = cat['item_id']

In [8]:
df = pd.merge(df,cat[['frn_id','item_code','category', 'item_name','item_description']],
              on=['frn_id','item_code'] ,how='inner', indicator=False)

KeyError: 'item_code'

In [9]:
df.loc[df.T2=='Tacos otros','T2'] = ['Tacos Otros' for i in range(sum(df.T2=='Tacos otros'))]

In [10]:
len(df)

2534

In [3]:
#df.T2.value_counts().plot(kind='bar', grid=True, figsize=(16, 9))

## Clean text and tokenize

In [4]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '´',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct == '<' or punct == '>':
            x = x.replace(punct, f' ')
        if punct in x:
            x = x.replace(punct, f' ')
            
    x = x.replace('\n', ' ')

    x = re.sub(r'\d+', ' ', x.lower()) # remove digits
    x = re.sub(r'\s\s+', ' ', x) # remove n-sized space n>1 
    
    return x

In [5]:
import unidecode
import string
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('es_core_news_md')
aditional_sw = set(['y','o','a','nan','oz','gr','ml',' ','  ', '   ', '    ',
                    '     ', '      ','....','paquete','promo','promocion','combo',
                    'promociones', 'promoción','exclusiva','persona',
                    'descuento', 'desc','oferta', 'rebaja', 
                    'menu','menú', 'comida','delicioso','deliciosa',
                    'rico','rica','especialidad', 'especialidades',
                    'favorito','favorita','personal','chica','grande',
                    'pequeña','chico','pequeño','individual','compartir'
                   ])
aditional_sw = aditional_sw.union(set([w+'s' for w in aditional_sw]))

stop_words = spacy.lang.es.stop_words.STOP_WORDS.union(aditional_sw) 

# Load tokenizer, tagger, parser, NER and word vectors
parser = Spanish()

# Creating our tokenizer function
def spacy_tokenizer(sentence):    
    # Creating our token object, which is used to create documents with linguistic annotations.
        

    mytokens = sentence.split(' ')

#     mytokens = [word.lemma_ for word in mytokens]
    
    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations and len(word)>2]
    
    # return preprocessed list of tokens
    return (mytokens)


In [6]:
def get_pos(text, which='VERB'):
    mytokens = nlp(text)
    return([token.text for token in mytokens if token.pos_ == which])

In [7]:
%%time
df_ = df

df_['item_text'] = df_['category'].map(str) + ' ' + df_['item_name'] + ' ' + df_['item_description'].map(str) 
df_['item_text'] = df_['item_text'].map(clean_text)


CPU times: user 1min 2s, sys: 999 ms, total: 1min 3s
Wall time: 1min 3s


In [8]:
%%time
df_['tokens'] = df_['item_text'].apply(spacy_tokenizer)

#df_['n_tokens'] = df_['tokens'].map(len)
df_['prep_text'] = df_['tokens'].map(' '.join)

CPU times: user 17.5 s, sys: 765 ms, total: 18.2 s
Wall time: 18.2 s


In [9]:
df_.to_pickle('whole_dish_base_2020.pkl')

In [12]:
df_['verbs'] = df_['item_text'].map(lambda x: get_pos(x.lower()))
df_['adjectives'] = df_['item_text'].map(lambda x: get_pos(x.lower(),'ADJ'))

## save working version of the dataset

In [1]:
#pd.read_pickle('taxonomy_working.pkl')

## EDA

In [21]:
df_.n_tokens.value_counts(sort=False).plot(kind='bar', grid=True, figsize=(16, 9))

AttributeError: 'DataFrame' object has no attribute 'n_tokens'

In [ ]:
def keywords(column = 'tokens',category = None):
    if category:
        tokens = df_[column][df_['T2'] == category]
    else:
        tokens = df_[column]
    alltokens = []
    #import pdb;pdb.set_trace()
    for token_list in tokens:
        alltokens.extend(token_list)
    counter = Counter(alltokens)
    return (counter.most_common(10))


In [ ]:
len(set(df_['T2']))

In [ ]:
# for source in set(df_['T2']):
#     print('T2 :', source)
#     print('top 10 keywords:', keywords(source))
#     print('---')

# forall
alltokens = []
for token_list in df_['verbs']:
    alltokens.extend(token_list)
    
counter = Counter(alltokens).most_common()


In [ ]:
counter[:20]